# NoisePy SCEDC Tutorial

Noisepy is a python software package to process ambient seismic noise cross correlations. This tutorial aims to introduce the use of noisepy for a toy problem on the SCEDC data. It can be ran locally or on the cloud.


The data is stored on AWS S3 as the SCEDC Data Set: https://scedc.caltech.edu/data/getstarted-pds.html



First, we install the noisepy-seis package

In [1]:
# Uncomment and run this line if the environment doesn't have noisepy already installed:
# ! pip install noisepy-seis 

__Warning__: NoisePy uses ```obspy``` as a core Python module to manipulate seismic data. If you use Google Colab, restart the runtime now for proper installation of ```obspy``` on Colab.

## Import necessary modules

Then we import the basic modules

In [2]:
from noisepy.seis import cross_correlate, stack, plotting_modules, __version__       # noisepy core functions
from noisepy.seis.asdfstore import ASDFCCStore, ASDFStackStore          # Object to store ASDF data within noisepy
from noisepy.seis.scedc_s3store import SCEDCS3DataStore, channel_filter # Object to query SCEDC data from on S3
from noisepy.seis.datatypes import ConfigParameters                     # Main configuration object
from noisepy.seis.channelcatalog import XMLStationChannelCatalog        # Required stationXML handling object
import os
from datetime import datetime
from datetimerange import DateTimeRange


from noisepy.seis.plotting_modules import plot_all_moveout

print(f"Using NoisePy version {__version__}")

path = "./scedc_data" 

os.makedirs(path, exist_ok=True)
cc_data_path = os.path.join(path, "CCF")
stack_data_path = os.path.join(path, "STACK")
S3_STORAGE_OPTIONS = {"s3": {"anon": True}}

/Users/marinedenolle/opt/miniconda3/envs/noisepy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using NoisePy version 0.9.72.dev12


We will work with a single day worth of data on SCEDC. The continuous data is organized with a single day and channel per miniseed (https://scedc.caltech.edu/data/cloud.html). For this example, you can choose any year since 2002. We will just cross correlate a single day.

In [3]:
# SCEDC S3 bucket common URL characters for that day.
S3_DATA = "s3://scedc-pds/continuous_waveforms/"
# timeframe for analysis
start = datetime(2002, 1, 1)
end = datetime(2002, 1, 3)
timerange = DateTimeRange(start, end)
print(timerange)

2022-01-01T00:00:00 - 2022-01-03T00:00:00


The station information, including the instrumental response, is stored as stationXML in the following bucket

In [4]:
S3_STATION_XML = "s3://scedc-pds/FDSNstationXML/CI/"            # S3 storage of stationXML


## Ambient Noise Project Configuration

We prepare the configuration of the workflow by declaring and storing parameters into the ``ConfigParameters()`` object and/or editing the ``config.yml`` file.


In [5]:
# Initialize ambient noise workflow configuration
config = ConfigParameters() # default config parameters which can be customized


Customize the job parameters below:

In [6]:

config.acorr_only = False  # only perform auto-correlation or not
config.xcorr_only = True  # only perform cross-correlation or not

config.inc_hours = 24 # INC_HOURS is used in hours (integer) as the 
        #chunk of time that the paralelliztion will work.
        # data will be loaded in memory, so reduce memory with smaller 
        # inc_hours if there are over 400+ stations.
        # At regional scale for SCEDC, we can afford 20Hz data and inc_hour 
        # being a day of data.

 
# pre-processing parameters
config.samp_freq= 20  # (int) Sampling rate in Hz of desired processing (it can be different than the data sampling rate)
config.cc_len= 3600.0  # (float) basic unit of data length for fft (sec)
config.step= 1800.0  # (float) overlapping between each cc_len (sec)

config.ncomp = 3  # 1 or 3 component data (needed to decide whether do rotation)

config.stationxml= False  # station.XML file used to remove instrument response for SAC/miniseed data
      # If True, the stationXML file is assumed to be provided.
config.rm_resp= "inv"  # select 'no' to not remove response and use 'inv' if you use the stationXML,'spectrum',


############## NOISE PRE-PROCESSING ##################
config.freqmin,config.freqmax = 0.05,2.0  # broad band filtering of the data before cross correlation
config.max_over_std  = 10  # threshold to remove window of bad signals: set it to 10*9 if prefer not to remove them

################### SPECTRAL NORMALIZATION ############
config.freq_norm= "rma"  # choose between "rma" for a soft whitening or "no" for no whitening. Pure whitening is not implemented correctly at this point.
config.smoothspect_N = 10  # moving window length to smooth spectrum amplitude (points)
    # here, choose smoothspect_N for the case of a strict whitening (e.g., phase_only)


#################### TEMPORAL NORMALIZATION ##########
config.time_norm = "one_bit"  # 'no' for no normalization, or 'rma', 'one_bit' for normalization in time domain,
config.smooth_N= 10  # moving window length for time domain normalization if selected (points)


############ cross correlation ##############

config.cc_method= "xcorr"  # 'xcorr' for pure cross correlation OR 'deconv' for deconvolution;
    # FOR "COHERENCY" PLEASE set freq_norm to "rma", time_norm to "no" and cc_method to "xcorr"

# OUTPUTS:
config.substack = True  # True = smaller stacks within the time chunk. False: it will stack over inc_hours
config.substack_len = config.cc_len  # how long to stack over (for monitoring purpose): need to be multiples of cc_len
    # if substack=True, substack_len=2*cc_len, then you pre-stack every 2 correlation windows.
    # for instance: substack=True, substack_len=cc_len means that you keep ALL of the correlations
    # if substack=False, the cross correlation will be stacked over the inc_hour window

### For monitoring applications ####
## we recommend substacking ever 2-4 cross correlations and storing the substacks
# e.g. 
# config.substack = True 
# config.substack_len = 4* config.cc_len

config.maxlag= 200  # lags of cross-correlation to save (sec)

In [7]:
# For this tutorial make sure the previous run is empty
os.system(f"rm -rf {cc_data_path}")


0

## Step 1: Cross-correlation

In this instance, we read directly the data from the scedc bucket into the cross correlation code. We are not attempting to recreate a data store. Therefore we go straight to step 1 of the cross correlations.

We first declare the data and cross correlation stores

In [8]:
#stations = "RPV,STS,LTP,LGB,WLT,CPP,PDU,CLT,SVD,BBR".split(",") # filter to these stations
stations = "RPV,SVD,BBR".split(",") # filter to these stations
# stations = "DGR,DEV,DLA,DNR,FMP,HLL,LGU,LLS,MLS,PDU,PDR,RIN,RIO,RVR,SMS,BBR,CHN,MWC,RIO,BBS,RPV,ADO,DEV".split(",") # filter to these stations
catalog = XMLStationChannelCatalog(S3_STATION_XML, storage_options=S3_STORAGE_OPTIONS) # Station catalog
raw_store = SCEDCS3DataStore(S3_DATA, catalog, channel_filter(stations, "BH"), timerange, storage_options=S3_STORAGE_OPTIONS) # Store for reading raw data from S3 bucket
cc_store = ASDFCCStore(cc_data_path,mode="w") # Store for writing CC data

get the time range of the data in the data store inventory

In [9]:
span = raw_store.get_timespans()
print(span)

[2022-01-01T00:00:00+0000 - 2022-01-02T00:00:00+0000, 2022-01-02T00:00:00+0000 - 2022-01-03T00:00:00+0000]


Get the channels available during a given time spane

In [10]:
channel_list=raw_store.get_channels(span[1])
print(channel_list)

2023-09-11 09:56:50,978 4314760576 INFO utils.log_raw(): TIMING: 2.8096 secs. for Loading 4035 files from s3://scedc-pds/continuous_waveforms/2022/2022_002/
2023-09-11 09:56:51,026 4314760576 INFO utils.log_raw(): TIMING: 0.0481 secs. for Init: 1 timespans and 24 channels
2023-09-11 09:56:51,125 11372883968 INFO channelcatalog._get_inventory_from_file(): Reading StationXML file s3://scedc-pds/FDSNstationXML/CI/CI_RPV.xml
2023-09-11 09:56:51,250 11490668544 INFO channelcatalog._get_inventory_from_file(): Reading StationXML file s3://scedc-pds/FDSNstationXML/CI/CI_LTP.xml
2023-09-11 09:56:51,252 11440189440 INFO channelcatalog._get_inventory_from_file(): Reading StationXML file s3://scedc-pds/FDSNstationXML/CI/CI_PDU.xml
2023-09-11 09:56:51,530 11423363072 INFO channelcatalog._get_inventory_from_file(): Reading StationXML file s3://scedc-pds/FDSNstationXML/CI/CI_CLT.xml
2023-09-11 09:56:51,531 11389710336 INFO channelcatalog._get_inventory_from_file(): Reading StationXML file s3://scedc-

[CI.BBR.BHE, CI.BBR.BHN, CI.BBR.BHZ, CI.CLT.BHE, CI.CLT.BHN, CI.CLT.BHZ, CI.LGB.BHE, CI.LGB.BHN, CI.LGB.BHZ, CI.LTP.BHE, CI.LTP.BHN, CI.LTP.BHZ, CI.PDU.BHE, CI.PDU.BHN, CI.PDU.BHZ, CI.RPV.BHE, CI.RPV.BHN, CI.RPV.BHZ, CI.STS.BHE, CI.STS.BHN, CI.STS.BHZ, CI.WLT.BHE, CI.WLT.BHN, CI.WLT.BHZ]


## Perform the cross correlation
The data will be pulled from SCEDC, cross correlated, and stored locally if this notebook is ran locally.
If you are re-calculating, we recommend to clear the old ``cc_store``.

In [11]:
cross_correlate(raw_store, config, cc_store)

2023-09-11 09:57:26,799 4314760576 INFO S1_fft_cc_MPI.cross_correlate(): Starting Cross-Correlation with 10 cores
2023-09-11 09:57:27,913 4314760576 INFO utils.log_raw(): TIMING: 1.1141 secs. for Loading 4071 files from s3://scedc-pds/continuous_waveforms/2022/2022_001/
2023-09-11 09:57:27,961 4314760576 INFO utils.log_raw(): TIMING: 0.0479 secs. for Init: 2 timespans and 48 channels
2023-09-11 09:57:27,962 4314760576 INFO scedc_s3store.get_channels(): Getting 24 channels for 2022-01-01T00:00:00+0000 - 2022-01-02T00:00:00+0000: [CI.BBR.BHE, CI.BBR.BHN, CI.BBR.BHZ, CI.CLT.BHE, CI.CLT.BHN, CI.CLT.BHZ, CI.LGB.BHE, CI.LGB.BHN, CI.LGB.BHZ, CI.LTP.BHE, CI.LTP.BHN, CI.LTP.BHZ, CI.PDU.BHE, CI.PDU.BHN, CI.PDU.BHZ, CI.RPV.BHE, CI.RPV.BHN, CI.RPV.BHZ, CI.STS.BHE, CI.STS.BHN, CI.STS.BHZ, CI.WLT.BHE, CI.WLT.BHN, CI.WLT.BHZ]
2023-09-11 09:57:27,969 4314760576 INFO utils.log_raw(): TIMING CC Main: 1.1696 secs. for get 24 channels
2023-09-11 09:57:27,970 4314760576 INFO S1_fft_cc_MPI.cross_correlate()

The cross correlations are saved as a single file for each channel pair and each increment of inc_hours. We now will stack all the cross correlations over all time chunk and look at all station pairs results.

## Step 2: Stack the cross correlation

We  now create the stack stores. Because this tutorial runs locally, we will use an ASDF stack store to output the data. ASDF is a data container in HDF5 that is used in full waveform modeling and inversion. H5 behaves well locally. 

Each station pair will have 1 H5 file with all components of the cross correlations. While this produces **many** H5 files, it has come down to the noisepy team's favorite option: 
1. the thread-safe installation of HDF5 is not trivial
2. the choice of grouping station pairs within a single file is not flexible to a broad audience of users.

In [12]:
# open a new cc store in read-only mode since we will be doing parallel access for stacking
cc_store = ASDFCCStore(cc_data_path, mode="r")
stack_store = ASDFStackStore(stack_data_path)

## Configure the stacking

There are various methods for optimal stacking. We refern to Yang et al (2022) for a discussion and comparison of the methods:

Yang X, Bryan J, Okubo K, Jiang C, Clements T, Denolle MA. Optimal stacking of noise cross-correlation functions. Geophysical Journal International. 2023 Mar;232(3):1600-18. https://doi.org/10.1093/gji/ggac410

Users have the choice to implement *linear*, phase weighted stacks *pws* (Schimmel et al, 1997), *robust* stacking (Yang et al, 2022), *acf* autocovariance filter (Nakata et al, 2019), *nroot* stacking. Users may choose the stacking method of their choice by entering the strings in ``config.stack_method``.

If chosen *all*, the current code only ouputs *linear*, *pws*, *robust* since *nroot* is less common and *acf* is computationally expensive.


In [13]:
config.stack_method = "linear"

In [14]:
method_list = [method for method in dir(cc_store) if method.startswith('__') is False]
print(method_list)


['_get_channel_pair', '_get_station_pair', 'append', 'contains', 'datasets', 'get_station_pairs', 'get_timespans', 'read_correlations']


In [15]:
cc_store.get_station_pairs()

[(CI.LGB, CI.STS), (CI.WLT, CI.WLT)]

In [16]:
stack(cc_store, stack_store, config)

2023-09-11 09:58:43,569 4314760576 INFO S2_stacking.initializer(): Station pairs: [(CI.LGB, CI.STS), (CI.WLT, CI.WLT)], timespans:[2022-01-01T00:00:00+0000 - 2022-01-02T00:00:00+0000, 2022-01-02T00:00:00+0000 - 2022-01-03T00:00:00+0000]
Stacking Pairs:   0%|          | 0/2 [00:00<?, ?it/s]2023-09-11 09:58:45,315 4384343424 WARNING asdfstore.read_correlations(): No data available for 2022-01-01T00:00:00+0000 - 2022-01-02T00:00:00+0000/CI.WLT_CI.WLT
2023-09-11 09:58:45,315 4384343424 WARNING S2_stacking.validate_pairs(): continue! not enough cross components for auto-correlation (CI.WLT, CI.WLT) in 2022-01-01T00:00:00+0000 - 2022-01-02T00:00:00+0000
2023-09-11 09:58:45,317 4384343424 WARNING S2_stacking.validate_pairs(): continue! not enough cross components for auto-correlation (CI.WLT, CI.WLT) in 2022-01-02T00:00:00+0000 - 2022-01-03T00:00:00+0000
2023-09-11 09:58:45,317 4384343424 INFO utils.log_raw(): TIMING: 0.0035 secs. for loading CCF data
Stacking Pairs. Memory:  2525 MB:  50%|██

## QC_1 of the cross correlations for Imaging
We now explore the quality of the cross correlations. We plot the moveout of the cross correlations, filtered in some frequency band.

In [17]:
cc_store.get_station_pairs()

[(CI.LGB, CI.STS), (CI.WLT, CI.WLT)]

In [18]:
plot_all_moveout(stack_store, 'Allstack_linear', 0.1, 0.2, 'ZZ', 1)

2023-09-11 09:58:45,344 4314760576 WARNING asdfstore.read(): Not data available for CI.LGB_CI.STS/Allstack_linear/ZZ
2023-09-11 09:58:45,345 4314760576 ERROR plotting_modules.plot_all_moveout(): No data available for plotting Allstack_linear/ZZ
